In [1]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
print("Done")

PyTorch Version: 2.6.0+cu118
CUDA Available: True
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 4090
Done


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# ✅ Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Load dataset CSV (ensure 'label' and 'image' columns are present)
csv_path = r"E:\Code\ISIC_2019_Training_GroundTruth_Transformed.csv"
df = pd.read_csv(csv_path)

# ✅ Add full image paths
image_folder = r"E:/Code/ISIC_2019_Training_Input"
df["image_path"] = df["image"].apply(lambda x: f"{image_folder}/{x}.jpg")

# ✅ Train-Validation Split (80-20)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["diagnosis"])

# ✅ Image Augmentation for training
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Validation Transform (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print("Done")

Using device: cuda
Done


In [3]:
import torch
from PIL import Image

# Label mapping for multiclass classification
label_mapping = {
    "MEL": 0,  # Melanoma
    "NV": 1,   # Melanocytic Nevus
    "BCC": 2,  # Basal Cell Carcinoma
    "AK": 3,   # Actinic Keratosis
    "BKL": 4,  # Benign Keratosis
    "DF": 5,   # Dermatofibroma
    "VASC": 6, # Vascular Lesion
    "SCC": 7   # Squamous Cell Carcinoma
}

class SkinLesionDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]["image_path"]
        label_str = self.dataframe.iloc[idx]["diagnosis"]
        
        # Convert string label to integer using the mapping
        label = label_mapping[label_str]

        # Convert to PyTorch tensor
        label = torch.tensor(label, dtype=torch.long)  

        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
# ✅ Create datasets
train_dataset = SkinLesionDataset(train_df, train_transform)
val_dataset = SkinLesionDataset(val_df, val_transform)

# ✅ Define batch size
batch_size = 64

# ✅ Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train size: {len(train_loader.dataset)}, Validation size: {len(val_loader.dataset)}")


Train size: 20264, Validation size: 5067


In [5]:
import torch.nn as nn

# Define SE Block (Channel Attention)
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)  # Global average pooling
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch, channels, _, _ = x.size()
        # Squeeze: Global average pooling
        y = self.squeeze(x).view(batch, channels)
        # Excitation: Fully connected layers to compute channel-wise weights
        y = self.excitation(y).view(batch, channels, 1, 1)
        # Scale the input feature map
        return x * y.expand_as(x)

# Define Hybrid CBAM + SE Block
class HybridAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(HybridAttention, self).__init__()
        # SE for channel attention
        self.se = SEBlock(channels, reduction)
        # Spatial Attention from CBAM
        self.spatial_att = nn.Sequential(
            nn.Conv2d(2, 1, 7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel attention with SE
        channel_att = self.se(x)
        # Spatial attention
        max_pool = torch.max(channel_att, dim=1, keepdim=True)[0]
        avg_pool = torch.mean(channel_att, dim=1, keepdim=True)
        spatial_att = torch.cat([max_pool, avg_pool], dim=1)
        spatial_att = self.spatial_att(spatial_att) * channel_att
        return spatial_att

print("Done")

Done


In [6]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights, densenet169, DenseNet169_Weights
class EnsembleModel(nn.Module):
    def __init__(self, num_classes=8):
        super(EnsembleModel, self).__init__()

        # Load Pretrained models
        self.efficientnet = efficientnet_b4(weights=EfficientNet_B4_Weights.IMAGENET1K_V1)
        self.densenet = densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)

        # Extract feature extractor part
        self.efficientnet = self.efficientnet.features  # EfficientNet feature extractor
        self.densenet = self.densenet.features  # DenseNet feature extractor

        # Hybrid Attention for individual models
        self.att_efficientnet = HybridAttention(1792)  # EfficientNet-b4 output channels
        self.att_densenet = HybridAttention(1664)  # DenseNet169 output channels

        # Pooling & Flattening
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()

        # Hybrid Attention after concatenation
        self.att_concat = HybridAttention(1792 + 1664)  # Combined features

        # Fully Connected layers
        self.fc = nn.Sequential(
            nn.Linear(1792 + 1664, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        # Feature extraction
        feat_a = self.att_efficientnet(self.efficientnet(x))
        feat_b = self.att_densenet(self.densenet(x))

        # Pool and flatten
        feat_a = self.flatten(self.pool(feat_a))
        feat_b = self.flatten(self.pool(feat_b))

        # Concatenate features
        combined = torch.cat((feat_a, feat_b), dim=1)

        # Apply hybrid attention
        combined = combined.unsqueeze(-1).unsqueeze(-1)  # Shape: [batch, 1792+1664, 1, 1]
        combined = self.att_concat(combined)
        combined = combined.squeeze(-1).squeeze(-1)  # Shape: [batch, 1792+1664]

        # Fully connected layers
        out = self.fc(combined)
        return out

print("Done")

Done


In [7]:
# ✅ Initialize model
num_classes = train_df['diagnosis'].nunique()
model = EnsembleModel(num_classes=num_classes)
model = model.to(device)

# ✅ Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

print("Model initialized and moved to GPU.")


Model initialized and moved to GPU.


In [8]:
from tqdm import tqdm

# ✅ Evaluation Function
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return total_loss / len(loader), accuracy

# ✅ Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        train_acc = 100 * correct / total
        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1}: Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}%")
print("Done")


Done


In [9]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50)


Epoch 1/50: 100%|███████████████████████████████████████████████████████| 317/317 [13:32<00:00,  2.56s/it, Loss=0.9673]


Epoch 1: Train Loss: 1.1033, Train Acc: 63.21%
Validation Loss: 0.8485, Validation Acc: 69.65%


Epoch 2/50: 100%|███████████████████████████████████████████████████████| 317/317 [13:25<00:00,  2.54s/it, Loss=1.0596]


Epoch 2: Train Loss: 0.8471, Train Acc: 69.68%
Validation Loss: 0.7128, Validation Acc: 73.73%


Epoch 3/50: 100%|███████████████████████████████████████████████████████| 317/317 [13:27<00:00,  2.55s/it, Loss=0.8308]


...

Epoch 48: Train Loss: 0.1762, Train Acc: 93.99%
Validation Loss: 0.6017, Validation Acc: 86.20%


Epoch 49/50: 100%|██████████████████████████████████████████████████████| 317/317 [13:30<00:00,  2.56s/it, Loss=0.2066]


Epoch 49: Train Loss: 0.1673, Train Acc: 94.22%
Validation Loss: 0.6156, Validation Acc: 85.45%


Epoch 50/50: 100%|██████████████████████████████████████████████████████| 317/317 [13:32<00:00,  2.56s/it, Loss=0.1889]


Epoch 50: Train Loss: 0.1724, Train Acc: 93.89%
Validation Loss: 0.5995, Validation Acc: 86.07%


In [26]:
from sklearn.metrics import f1_score, recall_score

def evaluate_with_metrics(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Collect predictions and labels for F1-score and Recall
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    accuracy = correct / total

    # Compute F1-score and Recall
    f1 = f1_score(all_labels, all_preds, average="weighted")
    recall = recall_score(all_labels, all_preds, average="weighted")

    return avg_loss, accuracy, f1, recall

# ✅ Example Usage
val_loss, val_acc, val_f1, val_recall = evaluate_with_metrics(model, val_loader, criterion)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
print(f"Validation F1-Score: {val_f1:.4f}, Validation Recall: {val_recall:.4f}")


Validation Loss: 0.5995, Validation Accuracy: 0.8607
Validation F1-Score: 0.8577, Validation Recall: 0.8607
